In [57]:
import numpy as np
from neural_network import MLP

In [2]:
from keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

print(f'X_train: {train_X.shape}')
print(f'Y_train: {train_y.shape}')
print(f'X_test:  {test_X.shape}')
print(f'Y_test:  {test_y.shape}')


X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [58]:
my_nn = MLP([28*28, 30, 30, 10])

In [4]:
train_x_flattened = [x.flatten().reshape((-1, 1)) for x in train_X]
test_x_flattened = [x.flatten().reshape((-1, 1)) for x in test_X]

In [5]:
train_y_vectorized = [ np.zeros((10, 1)) for _ in train_y]
for i, y in enumerate(train_y):
    train_y_vectorized[i][y] = 1.0
test_y_vectorized = [ np.zeros((10, 1)) for _ in test_y]
for i, y in enumerate(test_y):
    test_y_vectorized[i][y] = 1.0

In [59]:
my_nn.sgd(train_x_flattened[:10000], train_y_vectorized[:10000], test_x_flattened, test_y_vectorized, epochs=20, batch_size=100)

C:\Users\17033\Documents\gap_projects\backprop-practice\neural_network.py:12: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-1 * x))


Accuracy for epoch 0: 0.5002
Accuracy for epoch 1: 0.5593
Accuracy for epoch 2: 0.5966
Accuracy for epoch 3: 0.6408
Accuracy for epoch 4: 0.675
Accuracy for epoch 5: 0.683
Accuracy for epoch 6: 0.693
Accuracy for epoch 7: 0.6906
Accuracy for epoch 8: 0.6893
Accuracy for epoch 9: 0.6892
Accuracy for epoch 10: 0.6959
Accuracy for epoch 11: 0.7012
Accuracy for epoch 12: 0.7071
Accuracy for epoch 13: 0.7101
Accuracy for epoch 14: 0.7112
Accuracy for epoch 15: 0.714
Accuracy for epoch 16: 0.7222
Accuracy for epoch 17: 0.7256
Accuracy for epoch 18: 0.7281
Accuracy for epoch 19: 0.7291
